# Ninety-Nine Haskell Problems

Coming from [here](https://wiki.haskell.org/H-99:_Ninety-Nine_Haskell_Problems).


## Lists

### 1

 Find the last element of a list.

In [21]:
lastElement :: [a] -> a
lastElement [] = error "Empty list!"
lastElement [x] = x
lastElement (x:xs) = lastElement xs

### 2

Find the last but one element of a list.

In [22]:
lastButOneElement :: [a] -> a
lastButOneElement [] = error "Empty list!"
lastButOneElement [x] = error "Singleton!"
lastButOneElement [x, y] = x 
lastButOneElement (x:xs) = lastButOneElement xs 

### 3

Find the K'th element of a list. The first element in the list is number 1.

In [23]:
kthElement :: [a] -> Int -> a
kthElement _ 0 = error "First index is 1!"
kthElement [] _ = error "Not enough elements!"
kthElement (x:xs) 1 = x
kthElement (x:xs) n = kthElement xs (n-1)

### 4

Find the number of elements of a list.

In [24]:
numberElements :: [a] -> Int
numberElements [] = 0
numberElements (x:xs) = 1 + numberElements xs

### 5

Reverse a list.

In [25]:
reverseList :: [a] -> [a]
reverseList [] = []
reverseList (x:xs) = reverseList xs ++ [x]

### 6

Find out whether a list is a palindrome. A palindrome can be read forward or backward; e.g. (x a m a x).

In [26]:
isPalindrome x = x == reverseList x

### 7

Flatten a nested list structure.

A custom data type is needed for nested lists:

In [27]:
data NestedList a = Elem a | List [NestedList a]

In [28]:
flattenList :: NestedList a -> [a]
flattenList (Elem x) = [x]
flattenList (List (x:xs)) = (flattenList x) ++ (flattenList (List xs))

### 8

Eliminate consecutive duplicates of list elements.

In [29]:
elimConsec :: Eq a => [a] -> [a]
elimConsec [] = []
elimConsec [x] = [x]
elimConsec (x:y:xs)
    | x == y = elimConsec (x:xs)
    | otherwise = x: elimConsec (y:xs)

### 9

Pack consecutive duplicates of list elements into sublists. If a list contains repeated elements they should be placed in separate sublists.

In [30]:
packConsec :: Eq a => [a] -> [[a]]
packConsec [] = []
packConsec [x] = [[x]]
packConsec (x:xs)
    | x == (head (head pcxs)) = (x:(head pcxs)):(tail pcxs)
    | otherwise = [x]:pcxs
    where pcxs = packConsec xs

### 10

Run-length encoding of a list. Use the result of problem P09 to implement the so-called run-length encoding data compression method. Consecutive duplicates of elements are encoded as lists (N E) where N is the number of duplicates of the element E.

In [31]:
runLengthEncod :: Eq a => [a] -> [(Int, a)]
runLengthEncod x =
    let packed = packConsec x
        lengths = map numberElements packed  
        heads = map head packed 
    in  zip lengths heads  

## Lists, continued

### 11
Modify the result of problem 10 in such a way that if an element has no duplicates it is simply copied into the result list. Only elements with duplicates are transferred as (N E) lists.

In [32]:
data Encoded a = Single a | Multiple (Int, a) deriving (Show, Read, Eq)  

In [33]:
toEncod :: (Int, a) -> Encoded a
toEncod (1, x) = Single x
toEncod (n, x) = Multiple (n, x)

In [34]:
runLengthEncod' :: Eq a => [a] -> [Encoded a] 
runLengthEncod' x = map toEncod $ runLengthEncod x

### 12

Decode a run-length encoded list.

In [35]:
toDecod :: Eq a => Encoded a -> [a]
toDecod (Single x) = [x]
toDecod (Multiple (0, x)) = []
toDecod (Multiple (1, x)) = toDecod (Single x)
toDecod (Multiple (n, x)) = x:(toDecod (Multiple (n-1, x)))

In [36]:
runLengthDecod :: Eq a => [Encoded a] -> [a]
runLengthDecod x = concatMap toDecod x

### 13

Run-length encoding of a list (direct solution).

Implement the so-called run-length encoding data compression method directly. I.e. don't explicitly create the sublists containing the duplicates, as in problem 9, but only count them. As in problem P11, simplify the result list by replacing the singleton lists (1 X) by X.

In [37]:
runLengthEncod'' :: Eq a => [a] -> [(Int, a)]
runLengthEncod'' [] = []
runLengthEncod'' [x] = [(1, x)]
runLengthEncod'' (x:xs)
    | x == (snd hexs) = ((fst hexs) + 1, x):texs 
    | otherwise = (1, x):encodxs
    where encodxs = runLengthEncod'' xs
          hexs = head encodxs
          texs = tail encodxs

In [38]:
runLengthEncodDirect :: Eq a => [a] -> [Encoded a]
runLengthEncodDirect x = map toEncod (runLengthEncod'' x)

### 14

Duplicate the elements of a list.

In [39]:
repElem :: Int -> a -> [a]
repElem 0 _ = []
repElem n x = x:(repElem (n-1) x)

repElems :: Int -> [a] -> [a]
repElems 0 _ = []
repElems _ [] = []
repElems n (x:xs) = (repElem n x) ++ repElems n xs 

In [40]:
duplicateElems = repElems 2

### 15

Replicate the elements of a list a given number of times.

Done above.

### 16

Drop every N'th element from a list.

In [41]:
enumerate :: [a] -> [(Int, a)]
enumerate x =
    let numbers = [1..(numberElements x)]
    in zip numbers x

In [42]:
dropNthElement :: Int -> [a] -> [a]
dropNthElement n x = [snd enum_x | enum_x <- enumerate x, fst enum_x `mod` n /= 0]

### 17

Split a list into two parts; the length of the first part is given.

In [32]:
splitNElements :: Int -> [a] -> ([a], [a])
splitNElements 0 x = ([], x)
splitNElements _ [] = ([], [])
splitNElements n (x:xs) = (x:fst prevxs, snd prevxs)
    where prevxs = splitNElements (n-1) xs

### 18

Extract a slice from a list.

Given two indices, i and k, the slice is the list containing the elements between the i'th and k'th element of the original list (both limits included). Start counting the elements with 1.

In [44]:
sliceSubList :: Int -> Int -> [a] -> [a]
sliceSubList m n x = fst (splitNElements (n-m+1) (snd (splitNElements (m-1) x)))

### 19 
Rotate a list N places to the left.

Hint: Use the predefined functions length and (++).

In [45]:
rotateNPlaces :: Int -> [a] -> [a]
rotateNPlaces n x
    | n == 0 = x
    | n > 0 = snd splitnpos ++ fst splitnpos
    | n < 0 = snd splitnneg ++ fst splitnneg
    where splitnpos = splitNElements n x
          splitnneg = splitNElements (numberElements x + n) x

### 20

Remove the K'th element from a list.

In [46]:
removeNthElem :: Int -> [a] -> (a, [a])
removeNthElem n x = 
    let splitted = splitNElements (n-1) x 
        removed = head (snd splitted)
        result = fst splitted ++ tail (snd splitted)
    in (removed, result)

## Lists again

### 21

Insert an element at a given position into a list.

In [47]:
insertAtNth :: Int -> a -> [a] -> [a]
insertAtNth n x xs = 
    let splitted = splitNElements (n-1) xs
    in fst splitted ++ [x] ++ snd splitted

### 22

Create a list containing all integers within a given range.

In [48]:
rangeInt :: Int -> Int -> [Int]
rangeInt m n = 
    let total = [1..n]
        splitted = splitNElements (m-1) total 
    in snd splitted

### 23

Extract a given number of randomly selected elements from a list.

In [94]:
import System.Random 
gen <- newStdGen

In [76]:
pickRandom :: (RandomGen g) => g -> [a] -> (a, ([a], g))
pickRandom g x =
    let k = randomR (1, numberElements x) g
        elemandresidue = removeNthElem (fst k) x
    in (fst elemandresidue, (snd elemandresidue, snd k))

In [79]:
pickNRandom :: (RandomGen g) => g -> Int -> [a] -> [a]
pickNRandom g 0 _ = []
pickNRandom g n [] = []
pickNRandom g n x = elemrndm:(pickNRandom g' (n-1) residue)
    where baserandom = pickRandom g x
          elemrndm = fst baserandom
          g' = snd (snd baserandom)
          residue = fst (snd baserandom)

### 24

Lotto: Draw N different random numbers from the set 1..M.

In [92]:
lottoList :: (RandomGen g) => g -> Int -> Int -> [Int]
lottoList g m n = pickNRandom g m [1..n]

In [97]:
gen <- newStdGen
lottoList gen 3 10

[1,6,5]

### 25 

Generate a random permutation of the elements of a list.

In [98]:
randomPermutation :: (RandomGen g) => g -> [a] -> [a]
randomPermutation g x = pickNRandom g (numberElements x) x

In [102]:
gen <- newStdGen
randomPermutation gen "Javier"

"revJai"

### 26

Generate the combinations of K distinct objects chosen from the N elements of a list

In how many ways can a committee of 3 be chosen from a group of 12 people? We all know that there are C(12,3) = 220 possibilities (C(N,K) denotes the well-known binomial coefficients). For pure mathematicians, this result may be great. But we want to really generate all the possibilities in a list.

Something I tried and wasn't as helpful as I first thought:

In [36]:
matrioska :: [a] -> [([a], [a])]
matrioska x = [splitNElements i x | i <- [1..length x - 1]]

Simple recursion:

In [55]:
combinations :: Int -> [a] -> [[a]]
combinations 0 _ = [[]]
combinations _ [] = []
combinations n (x:xs) = [x:subcomb | subcomb <- combinations (n-1) xs] ++ (combinations n xs)